In [ ]:
import time
import pandas as pd
import requests
from google.cloud import bigquery
from datetime import datetime, timezone
from google.oauth2 import service_account

# Configurações iniciais
# Quem for testar deve colocar o caminho para a chave de serviço (arquivo .json) do seu próprio projeto
key_path = "CAMINHO/DA/SUA/CHAVE/service_account.json"  # Substitua com o caminho do arquivo de chave do seu projeto 
credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"]
)

# Inicializando cliente do BigQuery
bigquery_client = bigquery.Client(credentials=credentials, project=credentials.project_id)

# Configurações da carga dos dados
load_to_bigquery = True  # True para salvar no BQ e False para salvar localmente em CSV

def collect_data():
    url = "https://api.coingecko.com/api/v3/coins/markets"
    vs_currency = 'usd'
    per_page = 250  # O máximo que a API permite por página
    total_pages = 4  # O máximo de páginas para evitar erro 429

    all_data = []

    for page in range(1, total_pages + 1):
        params = {
            'vs_currency': vs_currency,
            'order': 'market_cap_desc',
            'per_page': per_page,
            'page': page,
            'sparkline': False
        }
        response = requests.get(url, params=params)
        response.raise_for_status()
        page_data = response.json()
        all_data.extend(page_data)
        print(f"Página {page}/{total_pages} coletada com sucesso ✅")
        time.sleep(1) # Pra evitar o limite da API antes da página 4
    
    df = pd.DataFrame(all_data)
    return df

def preprocess_data(df):
    df["timestamp_utc"] = datetime.now(timezone.utc)
    return df

def upload_to_bigquery(df, table_id):
    job_config = bigquery.LoadJobConfig(
        write_disposition="WRITE_APPEND",
        autodetect=True
    )
    job = bigquery_client.load_table_from_dataframe(df, table_id, job_config=job_config)
    job.result()
    print(f"Dados enviados para {table_id} com sucesso.✅")

# Função para inserir dados na tabela de criptomoedas
def insert_into_cryptocurrencies(df):
    cryptocurrencies_df = df[['id', 'name', 'symbol', 'market_cap_rank', 'last_updated']].copy()

    if load_to_bigquery:
        project_id = "SEU_ID_DE_PROJETO"  # Substitua pelo ID do seu projeto no GCP
        dataset_id = "SEU_DATASET_ID"  # Substitua pelo ID do seu dataset no BigQuery
        table_id = f"{project_id}.{dataset_id}.currencies"
        upload_to_bigquery(cryptocurrencies_df, table_id)
    else:
        cryptocurrencies_df.to_csv("cryptos_currencies.csv", index=False)
        print("⚠️ Dados de criptomoedas salvos localmente como CSV (cryptos_currencies.csv)")

# Função para inserir dados na tabela de preços
def insert_into_cryptocurrency_prices(df):
    prices_df = df[['id', 'current_price', 'market_cap', 'total_volume', 'timestamp_utc']].copy()

    if load_to_bigquery:
        project_id = "SEU_ID_DE_PROJETO"  # Substitua pelo ID do seu projeto no GCP
        dataset_id = "SEU_DATASET_ID"  # Substitua pelo ID do seu dataset no BigQuery
        table_id = f"{project_id}.{dataset_id}.currency_prices"
        upload_to_bigquery(prices_df, table_id)
    else:
        prices_df.to_csv("cryptos_prices.csv", index=False)
        print("⚠️ Dados de preços salvos localmente como CSV (cryptos_prices.csv)")

# Execução Principal
if __name__ == "__main__":
    print("Rodando no VSCode... 🚀")

    # Coletar e preparar os dados
    df = collect_data()
    df = preprocess_data(df)

    # Inserir nas tabelas
    insert_into_cryptocurrencies(df)
    insert_into_cryptocurrency_prices(df)

    print("✅ Script finalizado!")


Rodando no VSCode... 🚀
Página 1/4 coletada com sucesso ✅
Página 2/4 coletada com sucesso ✅
Página 3/4 coletada com sucesso ✅
Página 4/4 coletada com sucesso ✅


c:\Users\F44622675838\OneDrive - DIAGNOSTICOS DA AMERICA S.A\Área de Trabalho\Python\DASA\.venv\Lib\site-packages\google\cloud\bigquery\_pandas_helpers.py:489: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


Dados enviados para nav-prd.temp_dataset.teste_currencies com sucesso.✅


c:\Users\F44622675838\OneDrive - DIAGNOSTICOS DA AMERICA S.A\Área de Trabalho\Python\DASA\.venv\Lib\site-packages\google\cloud\bigquery\_pandas_helpers.py:489: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


Dados enviados para nav-prd.temp_dataset.teste_currency_prices com sucesso.✅
✅ Script finalizado!
